<center>
    <h1 style="font-family: Calibri; font-size: 34px; font-style: bold; text-transform: none; letter-spacing: 2.5px; background-color: #ffffff;">
        <b>HuBMAP + HPA - Hacking the Human Body</b>
    </h1>
    
</center>

<br>

<center>
<img src="https://i0.wp.com/sangerinstitute.blog/wp-content/uploads/2020/04/HCA-small-intestine-1440_540.jpg?fit=1440%2C540&ssl=1"/>
</center>

---

<p id="toc"></p>

<br><br>

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: blue; background-color: #ffffff;">TABLE OF CONTENTS</h1>

---

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;"><a href="#imports">0&nbsp;&nbsp;&nbsp;&nbsp;IMPORTS & GENERAL SETTINGS</a></h3>

---

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;"><a href="#background_information">1&nbsp;&nbsp;&nbsp;&nbsp;BACKGROUND INFORMATION</a></h3>

---


<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;"><a href="#helper_functions">2&nbsp;&nbsp;&nbsp;&nbsp;HELPER FUNCTIONS</a></h3>

---

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;"><a href="#create_dataset">3&nbsp;&nbsp;&nbsp;&nbsp;DATASET CREATION AND EXPLORATION</a></h3>

---

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;"><a href="#modelling">4&nbsp;&nbsp;&nbsp;&nbsp;MODELLING</a></h3>

---

<br>

<a id="imports"></a>

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; background-color: #ffffff; color: blue;" id="imports">0&nbsp;&nbsp;IMPORTS & GENERAL SETTINGS &nbsp;&nbsp;&nbsp;&nbsp;<a href="#toc">&#10514;</a></h1>

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

from PIL import Image

import torch
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

import albumentations as A
import cv2

!pip install monai
import monai
from monai.inferers import sliding_window_inference
from monai.metrics import DiceMetric
from monai.visualize import plot_2d_or_3d_image
from monai.data import decollate_batch

from monai.transforms import (
    Activations,
    AddChannel,
    AsDiscrete,
    Compose,
    LoadImage,
    RandRotate90,
    RandSpatialCrop,
    ScaleIntensity,
    EnsureType,
)

In [ ]:
# Global Settings

INPUT_PATH = "../input/hubmap-organ-segmentation"

# IMG Settings

IMG_SHAPE = (256, 256)
IMG_MEAN = [0.485, 0.456, 0.406]
IMG_STD = [0.229, 0.224, 0.225]

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

<br>

<a id="background_information"></a>

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: blue; background-color: #ffffff;" id="background_information">1&nbsp;&nbsp;BACKGROUND INFORMATION&nbsp;&nbsp;&nbsp;&nbsp;<a href="#toc">&#10514;</a></h1>

---

<h3 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: blue; background-color: #ffffff;">1.1 BASIC COMPETITION INFORMATION</h3>

---

<br><b style="text-decoration: underline; font-family: Verdana; text-transform: uppercase;">PRIMARY TASK DESCRIPTION</b>

<font color="red"><b>This is a placeholder copy/paste from the OVERVIEW>DESCRIPTION page of the Kaggle competition</b></font>

The Human BioMolecular Atlas Program (HuBMAP) is working to create a Human Reference Atlas at the cellular level. Sponsored by the National Institutes of Health (NIH), HuBMAP and Indiana University’s Cyberinfrastructure for Network Science Center (CNS) have partnered with institutions across the globe for this endeavor. A major partner is the Human Protein Atlas (HPA), a Swedish research program aiming to map the protein expression in human cells, tissues, and organs, funded by the Knut and Alice Wallenberg Foundation.

In this competition, you’ll identify and segment functional tissue units (FTUs) across five human organs. You'll build your model using a dataset of tissue section images, with the best submissions segmenting FTUs as accurately as possible.

If successful, you'll help accelerate the world’s understanding of the relationships between cell and tissue organization. With a better idea of the relationship of cells, researchers will have more insight into the function of cells that impact human health. Further, the Human Reference Atlas constructed by HuBMAP will be freely available for use by researchers and pharmaceutical companies alike, potentially improving and prolonging human life.

<h3 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: blue; background-color: #ffffff;">1.2 COMPETITION EVALUATION</h3>

---

<br><b style="text-decoration: underline; font-family: Verdana; text-transform: uppercase;">GENERAL EVALUATION INFORMATION</b>

This competition is evaluated on the mean Dice coefficient. The Dice coefficient can be used to compare the pixel-wise agreement between a predicted segmentation and its corresponding ground truth. The formula is given by:

<br><center><b style="font-size: 20px;">$Dice = \large\frac{2 * |X \cap Y|}{|X| + |Y|}$</b></center><br>

where X is the predicted set of pixels and Y is the ground truth. The Dice coefficient is defined to be 1 when both X and Y are empty. The leaderboard score is the mean of the Dice coefficients for each image in the test set.

<br><b style="text-decoration: underline; font-family: Verdana; text-transform: uppercase;">SUBMISSION FILE INFORMATION</b>

In order to reduce the submission file size, our metric uses run-length encoding on the pixel values. Instead of submitting an exhaustive list of indices for your segmentation, you will submit pairs of values that contain a start position and a run length. E.g. '1 3' implies starting at pixel 1 and running a total of 3 pixels (1,2,3).

Note that, at the time of encoding, the mask should be binary, meaning the masks for all objects in an image are joined into a single large mask. A value of 0 should indicate pixels that are not masked, and a value of 1 will indicate pixels that are masked.

The competition format requires a space delimited list of pairs. For example, '1 3 10 5' implies pixels 1,2,3,10,11,12,13,14 are to be included in the mask. The metric checks that the pairs are sorted, positive, and the decoded pixel values are not duplicated. The pixels are numbered from top to bottom, then left to right: 1 is pixel (1,1), 2 is pixel (2,1), etc.
The file should contain a header and have the following format:

```
id,rle
2,1 3
5,3 4
6,4 5 66 77
etc...
```

<br>

<h3 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: blue; background-color: #ffffff;">1.3 DATASET OVERVIEW</h3>

---

<br><b style="text-decoration: underline; font-family: Verdana; text-transform: uppercase;">GENERAL INFORMATION</b>

In this competition we are segmenting neuronal cells in images. The training annotations are provided as run length encoded masks, and the images are in PNG format. The number of images is small, but the number of annotated objects is quite high. The hidden test set is roughly 240 images.

<br><b style="text-decoration: underline; font-family: Verdana; text-transform: uppercase;">FILES</b>

[train/test].csv Metadata for the train/test set. Only the first few rows of the test set are available for download.

* `id` - The image ID.

* `organ` - The organ that the biopsy sample was taken from.

* `data_source` - Whether the image was provided by HuBMAP or HPA.

* `img_height` - The height of the image in pixels.

* `img_width` - The width of the image in pixels.
 
* `pixel_size` - The height/width of a single pixel from this image in micrometers. All HPA images have a pixel size of 0.4 µm. For HuBMAP imagery the pixel size is 0.5 µm for kidney, 0.2290 µm for large intestine, 0.7562 µm for lung, 0.4945 µm for spleen, and 6.263 µm for prostate.
 
* `tissue_thickness` - The thickness of the biopsy sample in micrometers. All HPA images have a thickness of 4 µm. The HuBMAP samples have tissue slice thicknesses 10 µm for kidney, 8 µm for large intestine, 4 µm for spleen, 5 µm for lung, and 5 µm for prostate.

* `rle` - The target column. A run length encoded copy of the annotations. Provided for the training set only.

* `age` - The patient's age in years. Provided for the training set only.

* `sex` - The sex of the patient. Provided for the training set only.
sample_submission.csv

* `id` - The image ID.

* `rle` - A run length encoded mask of the FTUs in the image.

<br>


<a id="helper_functions"></a>


<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: blue; background-color: #ffffff;" id="helper_functions">
    2&nbsp;&nbsp;HELPER FUNCTION&nbsp;&nbsp;&nbsp;&nbsp;<a href="#toc">&#10514;</a>
</h1>

---

In [ ]:
def rle_decode(mask_rle, shape, color=1):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background
    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0] * shape[1], dtype=np.float32)
    for lo, hi in zip(starts, ends):
        img[lo : hi] = color
    return img.reshape(shape).T

<br>


<a id="create_dataset"></a>


<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: blue; background-color: #ffffff;" id="create_dataset">
    3&nbsp;&nbsp;DATASET CREATION AND EXPLORATION&nbsp;&nbsp;&nbsp;&nbsp;<a href="#toc">&#10514;</a>
</h1>

---

<h3 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: blue; background-color: #ffffff;">3.0 Torch Dataset Class</h3>

---

In [ ]:
class HubMAP_Dataset(Dataset):
    def __init__(self, train):
        self.dataframe = pd.read_csv(os.path.join(INPUT_PATH, "train.csv"))
        
        # Stratified dataframe division by organ trait
        self.input = self.dataframe.groupby("organ", group_keys=False).apply(lambda x: x.sample(min(len(x), int(0.8*len(x)))))
        
        if not train:
            self.input = self.dataframe.drop(self.input.index)
            
        # Augmentations
        self.transform = A.Compose([
                                A.Resize(height = IMG_SHAPE[0], width = IMG_SHAPE[1]),
                                A.Normalize(mean=IMG_MEAN, std=IMG_STD)
                             ])
    
    def __len__(self):
        return self.input.shape[0]
    
    def __getitem__(self, ndx):
        # Image properties
        img_path = str(self.input.iloc[ndx, 0]) + ".tiff"
        shape = (int(self.input.iloc[ndx, 3]), int(self.input.iloc[ndx, 4]))
        
        # Image/Mask Loading
        img = Image.open(os.path.join(INPUT_PATH, "train_images", img_path))
        img = np.array(img)
        
        mask = rle_decode(self.input.iloc[ndx, -3], shape)
        
        aug_data = self.transform(image = img, mask = mask)
        
        return torch.tensor(np.moveaxis(aug_data["image"], 2, 0), dtype=torch.float32), torch.tensor(np.expand_dims(aug_data["mask"], axis=0), dtype=torch.int16)
        

<h3 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: blue; background-color: #ffffff;">3.1 VISUALIZE THE TRAIN DATA</h3>

---

In [ ]:
img = Image.open("../input/hubmap-organ-segmentation/train_images/10044.tiff")
img

In [ ]:
img = np.array(img)

df = pd.read_csv(os.path.join(INPUT_PATH, "train.csv"))
mask = rle_decode(df.iloc[0, -3], (3000, 3000))

plt.figure(figsize=(15, 15))
plt.imshow(img)
plt.imshow(mask, alpha=0.5)

<br>


<a id="modelling"></a>


<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: blue; background-color: #ffffff;" id="modelling">
    4&nbsp;&nbsp;MODELLING&nbsp;&nbsp;&nbsp;&nbsp;<a href="#toc">&#10514;</a>
</h1>

---

<h3 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: blue; background-color: #ffffff;">4.0 Train Loop</h3>

---

In [ ]:
# Model Elements

#model = monai.networks.nets.UNet(
#        spatial_dims=2,
#        in_channels=3,
#        out_channels=1,
#        channels=(16, 32, 64, 128, 256),
#        strides=(2, 2, 2, 2),
#        num_res_units=2,
#    ).to(device)

model = monai.networks.nets.SegResNet(
        spatial_dims = 2,
        in_channels=3,
        out_channels=1,
        upsample_mode = "deconv").to(device)

loss_function = monai.losses.DiceLoss(sigmoid=True)
optimizer = torch.optim.Adam(model.parameters(), 1e-3)

post_trans = Compose([EnsureType(), Activations(sigmoid=True), AsDiscrete(threshold=0.5)])
dice_metric = DiceMetric(include_background=True, reduction="mean", get_not_nans=False)

# Dataset
train_ds = HubMAP_Dataset(train=True)
train_loader = DataLoader(train_ds, batch_size=16)

val_ds = HubMAP_Dataset(train=False)
val_loader = DataLoader(val_ds, batch_size=4)

# Training Loop

val_interval = 2
best_metric = -1
best_metric_epoch = -1
num_epochs = 10
epoch_loss_values = list()
metric_values = list()
writer = SummaryWriter()
for epoch in range(num_epochs):
    print("-" * 10)
    print(f"epoch {epoch + 1}/{num_epochs}")
    model.train()
    epoch_loss = 0
    step = 0
    for batch_data in tqdm(train_loader):
        step += 1
        inputs, labels = batch_data[0].to(device), batch_data[1].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_len = len(train_ds) // train_loader.batch_size
        #print(f"{step}/{epoch_len}, train_loss: {loss.item():.4f}")
        writer.add_scalar("train_loss", loss.item(), epoch_len * epoch + step)
    epoch_loss /= step
    epoch_loss_values.append(epoch_loss)
    print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}")

    if (epoch + 1) % val_interval == 0:
        model.eval()
        with torch.no_grad():
            val_images = None
            val_labels = None
            val_outputs = None
            for val_data in tqdm(val_loader):
                val_images, val_labels = val_data[0].to(device), val_data[1].to(device)
                roi_size = (96, 96)
                sw_batch_size = 4
                val_outputs = sliding_window_inference(val_images, roi_size, sw_batch_size, model)
                val_outputs = [post_trans(i) for i in decollate_batch(val_outputs)]
                # compute metric for current iteration
                dice_metric(y_pred=val_outputs, y=val_labels)
            # aggregate the final mean dice result
            metric = dice_metric.aggregate().item()
            # reset the status for next validation round
            dice_metric.reset()
            metric_values.append(metric)
            if metric > best_metric:
                best_metric = metric
                best_metric_epoch = epoch + 1
                torch.save(model.state_dict(), "best_metric_model_segmentation2d_array.pth")
                print("saved new best metric model")
            print(
                "current epoch: {} current mean dice: {:.4f} best mean dice: {:.4f} at epoch {}".format(
                    epoch + 1, metric, best_metric, best_metric_epoch
                )
            )
            writer.add_scalar("val_mean_dice", metric, epoch + 1)
            # plot the last model output as GIF image in TensorBoard with the corresponding image and label
            plot_2d_or_3d_image(val_images, epoch + 1, writer, index=0, tag="image")
            plot_2d_or_3d_image(val_labels, epoch + 1, writer, index=0, tag="label")
            plot_2d_or_3d_image(val_outputs, epoch + 1, writer, index=0, tag="output")

<h3 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: blue; background-color: #ffffff;">4.1 Trained Model Results</h3>

---

In [ ]:
for idx,batch in enumerate(val_loader):
    if idx == 0:
        img, mask = batch
    else:
        break

In [ ]:
#model = monai.networks.nets.UNet(
#        spatial_dims=2,
#        in_channels=3,
#        out_channels=1,
#        channels=(16, 32, 64, 128, 256),
#        strides=(2, 2, 2, 2),
#        num_res_units=2,
#    )

model = monai.networks.nets.SegResNet(
        spatial_dims = 2,
        in_channels=3,
        out_channels=1,
        upsample_mode = "deconv")

model.load_state_dict(torch.load("./best_metric_model_segmentation2d_array.pth"))

out = model(img)

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(15, 15))

for i in range(2):
    for j in range(2):
        if i == 1:
            aux = 1
        else:
            aux = 0
            
        ax[i,j].imshow(img[0+i+j+aux, 0, ...], cmap="gray")
        ax[i,j].imshow(out[0+i+j+aux, 0, ...].detach().numpy(), cmap="Greens", alpha = 0.5)
        ax[i,j].imshow(mask[0+i+j+aux, 0, ...], cmap="Blues", alpha = 0.3)